In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
class NoisyBatchNormalization(tf.keras.layers.BatchNormalization):
    
    def call(self, inputs, training, alpha):
        input_shape = K.int_shape(inputs)
        # Prepare broadcasting shape.
        ndim = len(input_shape)
        reduction_axes = list(range(len(input_shape)))
        del reduction_axes[self.axis]
        
        mean, var = tf.nn.moments(inputs,axes=reduction_axes)